In [1]:
import pandas as pd
import numpy as np
import os
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import string

In [25]:
path = os.path.join(os.path.dirname(os.getcwd()),'raw_data')
df = pd.read_csv(os.path.join(path,'bbc_final_df.csv'))

In [26]:
df.reset_index(drop=True, inplace=True)

In [27]:
df = df[df['stars']!='n']

In [28]:
df = df.sort_values(by='stars', ascending=False)

In [29]:
df = df[df['preference']!='n']

,recipe_title,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
2381,Spiced cauliflower roast,vegetarian,4.9,20,40,18.0,Easy,Serves 4 - 6,Cauliflower is having a renaissance and we cou...,"50g ,butter,, softened at room temperature, 2 ...","butter, chilli flakes, sumac, ground cumin, ca...",british,"['butter', 'chilli', 'sumac', 'cumin', 'caulif..."
1788,"Pea, feta & quinoa spring rolls with roast tom...",vegetarian,4.9,40,2 hr,8.0,A challenge,Makes 12,Serve up these vegetarian filo pastry canapés ...,"50g ,quinoa, 200g ,frozen petits pois, 85g ,fe...","quinoa, feta cheese, small bunch mint, spring ...",thai,"['quinoa', 'feta', 'cheese', 'mint', 'spring',..."
1799,"Feta, avocado & red pepper salad with honey-li...",vegetarian,4.9,15,n,10.0,Easy,Serves 4,"Light and filling, packed full of healthy summ...","200g ,feta cheese, 2 , ripe avocados, 2 ,red p...","feta cheese, ripe avocados, red peppers, can...",greek,"['feta', 'cheese', 'avocados', 'peppers', 'can..."
5447,Apricot & ginger ham,n,4.9,10,3 hrs and 30,10.0,More effort,Serves 15 - 20,"Glaze your gammon joint with fruit, bay leaves...","4-5kg raw smoked ham, on the bone (ask your b...","bottle ginger, 30-40 whole cloves, ground gin...",british,"['ginger', 'cloves', 'ginger']"
10317,Spiced lamb with bulgur & tahini dressing,n,4.9,15,25,10.0,Easy,Serves 2,Flavour lamb mince with cumin and coriander an...,"2 ½ tbsp ,olive oil, 2 ,onions,, halved and th...","olive oil, onions, garlic cloves, red chilli, ...",middle-eastern,"['olive', 'oil', 'onion', 'garlic', 'cloves', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352,Coconut Nice,vegan,1.0,30,40,1.0,Easy,Makes around 30,Upgrade the classic Nice biscuit with a coconu...,"1 tbsp ,linseeds, 400g ,plain flour,, plus ext...","linseeds, plain flour, coconut oil, desiccated...",british,"['linseed', 'flour', 'coconut', 'oil', 'desicc..."
1351,Coconut Nice,vegetarian,1.0,30,40,1.0,Easy,Makes around 30,Upgrade the classic Nice biscuit with a coconu...,"1 tbsp ,linseeds, 400g ,plain flour,, plus ext...","linseeds, plain flour, coconut oil, desiccated...",british,"['linseed', 'flour', 'coconut', 'oil', 'desicc..."
494,Smoked paprika & onion potato cake,vegetarian,1.0,20,40,2.0,Easy,Serves 6,Try this potato cake steeped in Spanish flavou...,"100g ,butter, 2 ,onions,, peeled and finely sl...","butter, onions, 2 tsp smoked paprika, manchego,",spanish,"['butter', 'onion', 'paprika', 'manchego']"
10841,Green Thai noodle broth,n,1.0,5,n,2.0,Easy,Serves 2,Whip up this cheat's supper in just 5 minutes ...,"600g ,pot green Thai chicken soup, (I used Wai...","beansprouts,",thai,['beansprouts']


In [30]:
dfdouble = df.sort_values(by='review_count', ascending=False)

In [31]:
dfdouble

,recipe_title,dietary,stars,prep_times,cooking_times,review_count,difficulty_level,servings,description,specific_ingredients,ingredients,preference,final_ingredients
5238,Mini sausage rolls,n,4.7,20,25,99.0,Easy,Makes 20,Everybody's favourite party food. John Torode'...,"½ small ,garlic clove, handful of ,parsley,, ...","garlic clove, parsley, beaten egg,",british,"['garlic', 'clove', 'parsley', 'egg']"
1566,Romantic rose cupcakes,vegetarian,4.8,10,20,99.0,Easy,Makes 12 deep cupcakes,These decorative cupcakes make the perfect cen...,"150ml ,pot natural yogurt, 3 ,eggs,, beaten, ...","pot natural yogurt, eggs, vanilla extract, go...",british,"['natural', 'yogurt', 'egg', 'vanilla', 'sugar..."
5219,Beetroot & chocolate cake,n,3.6,15,1 hr,99.0,Easy,Cuts into 8 slices,Beetroot is actually very sweet and keeps this...,"1 ,large cooked beetroot,, about 175g in weigh...","large cooked beetroot, plain flour, baking pow...",british,"['beetroot', 'flour', 'powder', 'sugar', 'egg'..."
6214,"Walnut, date & honey cake",n,4.3,15,1 hr and 10,99.0,Easy,Cuts into 8-10 slices,A tea-time treat - packed with sweet honey. A ...,"225g ,self-raising flour, ½ tsp ground cinnamo...","self-raising flour, ½ tsp ground cinnamon, sof...",british,"['selfraising', 'flour', 'cinnamon', 'butter',..."
10736,Thai chicken cakes with sweet chilli sauce,n,4.0,25,n,98.0,Easy,Serves 2 - 3,A light and healthy supper that's a great vari...,"2 ,large boneless, skinless chicken breasts, ...","large boneless, skinless chicken breasts, gar...",thai,"['chicken', 'breasts', 'garlic', 'clove', 'oni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9569,Coffee & cream cocktail,n,4.0,10,n,1.0,Easy,Makes 8,This coffee and cream layered cocktail makes a...,"400ml ,Tia Maria,, chilled, 16 tbsp ,(approx 2...","(approx 250ml) double cream,",italian,"['approx', 'cream']"
7712,Watermelon cooler,n,4.0,10,n,1.0,Easy,Serves 6,A simple fruity jelly that makes a gorgeous an...,"1 ,watermelon,, half chopped, half cut into bi...","watermelon, juice of 2 limes, white caster su...",british,"['watermelon', 'limes', 'sugar', 'gelatine', '..."
753,Purple sprouting broccoli with Parmesan & herb...,vegetarian,4.0,15,20,1.0,Easy,Serves 6,A superhealthy accompaniment to your Sunday roast,"25g ,butter, 4 tbsp ,olive oil, 50g ,white bre...","butter, olive oil, chopped thyme leaves, grate...",british,"['butter', 'olive', 'oil', 'thyme', 'parmesan'..."
7278,Lighter cauliflower cheese,n,4.0,20,30,1.0,Easy,Serves 4,Makeover this classic side dish with some clev...,"400ml ,semi-skimmed milk, 2 tbsp ,cornflour, 2...","semi-skimmed milk, garlic cloves, large caulif...",british,"['milk', 'garlic', 'cloves', 'cauliflower', 'c..."


In [21]:
df[df['recipe_title'] == 'Pork & sauerkraut goulash'].index.tolist()[0]

10882

In [5]:
def model_specific_preprocessing(tokenized_sentence):
    stop_words = set(stopwords.words('english'))
    no_stopwords = [w for w in tokenized_sentence if not w in stop_words]
    verb_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "v") for tok in no_stopwords]
    noun_lemmatized = [WordNetLemmatizer().lemmatize(tok, pos = "n")for tok in verb_lemmatized]
    ingredient_string = ' '.join(noun_lemmatized)
    return ingredient_string

def ingredients_list(sample_txt):
    elements = []
    for punc in string.punctuation:
        sample_txt = sample_txt.replace(punc,'')
    for word in sample_txt.split():
        elements.append(str(word))
    return elements

In [6]:
def k_means(bbc_final_df, clusters=75, min_df=0.00001, max_df=0.3):
    """Returns the df inputted with an additional column with their respective cluster.
    Ensure that the ingredients columns is a string of a list called 'final_ingredients'.
    I.E row 1, col ingredient is '[bread,cheese,pasta,onions]'"""

    # Step 1: Filter out recipes with no names & remove duplicates, keeping dietary info
    filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
    grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
    grouped_df.rename(columns = {'dietary':'combined'}, inplace=True)
    merged_df = grouped_df.merge(filtered_df,how='left',on='recipe_title')
    dropped_df = merged_df.drop(columns = 'dietary')
    df = dropped_df.drop_duplicates('recipe_title')

    # Step 2: Convert the final ingredients into a "sentence" for vectorising
    df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
    df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))

    # Step 3: Convert ingredients sentence into a bag-of-words representation
    vectorizer = CountVectorizer(min_df=min_df, max_df = max_df)
    counted_words = pd.DataFrame(vectorizer.fit_transform(df['clean_text']).toarray(), columns = vectorizer.get_feature_names_out())

    # Step 4: Apply KMeans clustering
    kmeans = KMeans(n_clusters=clusters, random_state=0)
    df['cluster'] = kmeans.fit_predict(counted_words)

    return df

In [7]:
df1 = k_means(df)

/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_86427/446812014.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_86427/446812014.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text']=df['ingredients_list'].map(lambda x: model_specific_preprocessing(x))
/var/folders/rp/7ftqrxx9701fy2jzfscnlwyw0000gn/T/ipykernel_86427/446812014.py:24: SettingWithCop

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
ingredients_cluster = df1[df1['cluster']==2][['recipe_title','clean_text','cluster']]


In [10]:
ingredients_cluster = ingredients_cluster.reset_index(drop=True)

In [11]:
ingredients_cluster_list = ingredients_cluster.clean_text.tolist()
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(ingredients_cluster_list)
similarity = cosine_similarity(vectors)

In [12]:
similar= pd.DataFrame(similarity)

In [13]:
similar

,0,1,2,3,4,5,6,7,8,9,...,299,300,301,302,303,304,305,306,307,308
0,1.000000,0.113908,0.290698,0.008992,0.010385,0.037516,0.007837,0.020518,0.011418,0.187231,...,0.008522,0.005942,0.012220,0.008406,0.083073,0.058792,0.100191,0.007026,0.009528,0.400792
1,0.113908,1.000000,0.683737,0.010554,0.012189,0.044034,0.182361,0.024083,0.013402,0.009686,...,0.010003,0.006974,0.014343,0.009867,0.097507,0.348151,0.117599,0.008247,0.011184,0.044561
2,0.290698,0.683737,1.000000,0.012110,0.013986,0.050525,0.209241,0.027633,0.015378,0.011114,...,0.011478,0.008002,0.016457,0.011321,0.111880,0.399467,0.134933,0.009463,0.012832,0.184442
3,0.008992,0.010554,0.012110,1.000000,0.559555,0.042005,0.045154,0.022973,0.012785,0.180413,...,0.009542,0.224477,0.070403,0.048432,0.016394,0.011602,0.015138,0.040481,0.010668,0.007492
4,0.010385,0.012189,0.013986,0.559555,1.000000,0.048512,0.131377,0.026532,0.014765,0.291783,...,0.011021,0.379381,0.233854,0.140915,0.018934,0.013400,0.017483,0.117781,0.012321,0.008653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,0.058792,0.348151,0.399467,0.011602,0.013400,0.048407,0.010112,0.026474,0.014733,0.010648,...,0.010997,0.007666,0.015767,0.010847,0.107190,1.000000,0.017445,0.009066,0.012294,0.173145
305,0.100191,0.117599,0.134933,0.015138,0.017483,0.063159,0.013194,0.034542,0.019223,0.013893,...,0.014348,0.010003,0.020572,0.014152,0.024650,0.017445,1.000000,0.011829,0.016041,0.011265
306,0.007026,0.008247,0.009463,0.040481,0.117781,0.032822,0.254125,0.017951,0.009990,0.063662,...,0.007456,0.175723,0.113899,0.064850,0.012810,0.009066,0.011829,1.000000,0.008336,0.005854
307,0.009528,0.011184,0.012832,0.010668,0.012321,0.044510,0.009298,0.024343,0.181180,0.009791,...,0.010111,0.007049,0.014498,0.009973,0.017372,0.012294,0.016041,0.008336,1.000000,0.007939


In [17]:
#def similarity_cal(index):
similar[0].sort_values(ascending=False)

0      1.000000
213    0.461546
308    0.400792
290    0.388037
201    0.366353
         ...   
278    0.006600
271    0.006480
107    0.006100
285    0.006018
300    0.005942
Name: 0, Length: 309, dtype: float64

In [15]:
ingredients_cluster['sim']=similar[0]

In [16]:
ingredients_cluster.sort_values(by='sim', ascending=False)

,recipe_title,clean_text,cluster,sim
0,Apple & walnut marmalade,dessert apple butter muscovado sugar lemon bra...,2,1.000000
213,Quick-to-mix mincemeat,bramley apple orange lemon muscovado sugar arm...,2,0.461546
308,Zesty mincemeat,muscovado sugar orange lemon coarsley bramley ...,2,0.400792
290,Traditional mincemeat,brandy lemon sugar nutmeg bramley apple,2,0.388037
201,Plum pudding,selfraising flour muscovado sugar orange lemon...,2,0.366353
...,...,...,...,...
278,"Steak, roasted pepper & pearl barley salad",barley pepper pepper onion olive oil steak wat...,2,0.006600
271,Spring roast chicken,chicken thigh olive oil lemon shallot rosemary...,2,0.006480
107,Griddled chicory with figs & bitter leaves,head chicory lemon olive oil fig handful mizun...,2,0.006100
285,Superhealthy salmon salad,couscous olive oil salmon sprout broccoli lemo...,2,0.006018


In [19]:
ingredients_cluster.head(3).clean_text

0    dessert apple butter muscovado sugar lemon bra...
1             apricot preserve sugar lemon knob butter
2            apricot apple preserve sugar lemon butter
Name: clean_text, dtype: object

In [ ]:
p= pd.DataFrame(ingredients_cluster)

In [ ]:
p =p.sort_values(by='sim', ascending=False)

In [ ]:
p.head(15)

In [ ]:
def ingredients_list(sample_txt):
    elements = []
    for punc in string.punctuation:
        sample_txt = sample_txt.replace(punc,'')
    for word in sample_txt.split():
        elements.append(str(word))
    return elements

def remove_most_common_ings(ing_list):
    too_common = ['oil', 'butter', 'salt', 'clove', 'cloves']
    no_stopwords = [w for w in ing_list if not w in too_common]
    return ' '.join(no_stopwords)

In [ ]:
df['ingredients_list'] = df['final_ingredients'].apply(ingredients_list)
df['temp_nocommon'] = df['ingredients_list'].apply(remove_most_common_ings)
df

# dsfjadsfasd

In [ ]:
ls

In [ ]:
dff = pd.read_csv('bbc_final_df.csv', index_col=0)

In [ ]:
bbc_final_df = dff.copy()
bbc_final_df

In [ ]:
filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
filtered_df

In [ ]:
filtered_df = bbc_final_df[bbc_final_df['recipe_title']!='n']
grouped_df = filtered_df.groupby('recipe_title')['dietary'].apply(lambda x: ', '.join(x)).reset_index().rename(columns = {'dietary':'combined'})
#grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
merged_df = grouped_df.merge(filtered_df, how='left',on='recipe_title')
dropped_df = merged_df.drop(columns = 'dietary')
df = dropped_df.drop_duplicates('recipe_title')

In [ ]:
df

In [ ]:
set(df.combined.tolist())

In [ ]:
df.tail(20)

In [ ]:
df['combined'].replace('n','None')

In [ ]:
cuisine_pref = 'american'
diet_pref = 'vegetarian'

In [ ]:
if cuisine_pref and diet_pref == 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    # st.write(star_sorted_df)
    print(star_sorted_df)

if cuisine_pref == 'Select' and diet_pref != 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df[star_sorted_df['combined'].contains(diet_pref)]
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    print(star_sorted_df)

    # st.write(star_sorted_df)
if diet_pref == 'Select'and cuisine_pref != 'Select':
    star_sorted_df = processed_df[processed_df['stars']!='n']
    star_sorted_df = star_sorted_df[star_sorted_df['preference']==cuisine_pref]
    star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
    print(star_sorted_df)


In [ ]:
star_sorted_df = processed_df[processed_df['stars']!='n']
star_sorted_df[star_sorted_df['preference']==cuisine_pref]
star_sorted_df = star_sorted_df[~star_sorted_df.recipe_title.duplicated()]
star_sorted_df.reset_index(drop=True,inplace=True)
star_sorted_df.sort_values(by='stars', ascending=False)

In [ ]:
df22 = pd.read_csv('bbc_final_df.csv')

In [ ]:
star_sorted_df = df22.copy()
star_sorted_df=star_sorted_df[star_sorted_df['stars']!='n']

In [ ]:
filtered_df = df22[df22['recipe_title'] != 'n']

grouped_df = filtered_df.groupby('recipe_title')['dietary'].apply(lambda x: ', '.join(x)).reset_index().rename(columns = {'dietary':'combined'})
# grouped_df = filtered_df.groupby('recipe_title').sum()[['dietary']].reset_index()
# grouped_df.rename(columns = {'dietary':'combined'}, inplace=True)
merged_df = grouped_df.merge(filtered_df, how='left',on='recipe_title')
# dropped_df = merged_df.drop(columns = 'dietary')
df = merged_df.drop_duplicates('recipe_title')



In [ ]:
df